# PyTorch를 사용한 CNN 모델 설계 및 학습 튜토리얼
이 튜토리얼은 Street View House Numbers (SVHN) 데이터를 사용하여 PyTorch를 기반으로한 Convolutional Neural Network (CNN) 모델을 설계하고 학습시키는 과정을 안내합니다. 아래는 튜토리얼의 필수 항목입니다.

#  1. PyTorch 및 데이터 로드 PyTorch 및 SVHN 데이터를 다운로드하고 로드하기 위한 라이브러리를 설치합니다.

In [ ]:
//bash 
pip install torch torchvision matplotlib


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import matplotlib.pyplot as plt


# SVHN 데이터를 로드합니다.

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


# 2. 모델 설계:  간단한 CNN 모델을 설계합니다.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()


# 3. 학습 : 모델을 정의하고, 손실 함수 및 옵티마이저를 선택한 후 학습합니다.

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):  # 10 epochs
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print("Training complete")


# 4. 테스트 : 학습된 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test accuracy: {accuracy}')


# 5. 평가방법에 대한 소개 학습된 모델의 성능을 더 자세히 평가하고 개선 방안을 모색합니다. 정확도 외에 다른 평가 지표도 고려할 수 있습니다.

# 6. 개선방안 모델 성능을 향상시키기 위한 다양한 방법을 탐구합니다. 레이어 추가, 하이퍼파라미터 조정 등을 고려할 수 있습니다.

# 7. 모델 저장 : 학습된 모델을 저장합니다.

In [ ]:
torch.save(model.state_dict(), 'svhn_model.pth')


# 8. 저장된 모델 불러와서 테스트 : 저장된 모델을 불러와 새로운 데이터에 대해 테스트합니다.

In [ ]:
model = CNN()
model.load_state_dict(torch.load('svhn_model.pth'))
model.eval()

# 이후 테스트 데이터에 대한 예측 수행


# SVHN 데이터에 대한 PyTorch 기반의 CNN 모델을 설계, 학습, 평가하고 저장, 불러와서 테스트하는 기본적인 과정을 설명 추가적인 개선 및 실험을 통해 모델을 향상 가능합니다.